# Phase 5: Gemini API 기반 인기글 생성 및 고도화

## 📌 목적
Phase 2에서 추출한 **팩터(인기유형)**, **피처(인기요소)**, **키워드** 정보를
Gemini API 프롬프트에 통합하여 서담 스타일 인기글을 생성합니다.
또한, 기존 Phase 4에서 생성된 결과물을 Gemini를 통해 더 자연스럽게 교정(Refining)합니다.

### 핵심 구조
1. **신규 생성**: 팩터 + 피처 + 키워드 → Gemini 프롬프트 → Thinking(전략) → Title + Content
2. **결과 재처리 (Refining)**: Phase 4 Llama 생성물 → Gemini 교정 → 고품질 인기글

### 사전 조건
- `phase2_dataset_v2.ipynb` 실행 완료 (`outputs/rpm_factors.json` 등 존재)
- (선택) `phase4_generation_v2.ipynb` 실행 완료 (`outputs/generated_posts_v2.csv` 존재)
- Colab Secrets에 `GEMINI_API_KEY` 등록
- GPU **불필요** (API 기반)

---
## 0. 환경 설정

In [ ]:
from google.colab import drive, userdata
drive.mount('/content/drive', force_remount=True)

import google.generativeai as genai
import json, time, re, random
from pathlib import Path
from datetime import datetime
import pandas as pd
from IPython.display import display
from tqdm.auto import tqdm

# ── Gemini API 연결 ─────────────────────────────────────
try:
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)
    gemini_model = genai.GenerativeModel('gemini-2.0-flash')
    print("✅ Gemini API 연결 완료")
except Exception as e:
    print(f"❌ API 연결 실패: {e}")
    print("💡 Colab 왼쪽 열쇠 아이콘(Secrets)에서 GEMINI_API_KEY를 등록했는지 확인하세요.")

---
## 1. RPM 데이터 로드 (팩터 + 피처 + 키워드)

In [ ]:
PROJECT_ROOT = Path("/content/drive/MyDrive/board_crawling")
OUTPUT_DIR   = PROJECT_ROOT / "outputs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# ── 팩터 정보 ────────────────────────────────────────────
FACTORS_PATH = OUTPUT_DIR / "rpm_factors.json"
if FACTORS_PATH.exists():
    with open(FACTORS_PATH, "r", encoding="utf-8") as f:
        factors_data = json.load(f)
    factor_names = [f["factor_name"] for f in factors_data["factors"]]
    factor_descriptions = {
        f["factor_name"]: f.get("description", "")
        for f in factors_data["factors"]
    }
    factor_stats = factors_data.get("factor_stats", {})
    print(f"✅ 팩터 로드: {len(factor_names)}개")
else:
    factor_names = ["감정적_공감", "유머_반전", "정보_전달", "참여_유도", "일상_공유", "논란_이슈"]
    factor_descriptions = {n: "" for n in factor_names}
    factor_stats = {}
    print("⚠️ 팩터 파일 없음 → 기본 팩터 사용")

# ── 피처 정보 ────────────────────────────────────────────
FEATURES_PATH = OUTPUT_DIR / "rpm_features.json"
if FEATURES_PATH.exists():
    with open(FEATURES_PATH, "r", encoding="utf-8") as f:
        all_features = json.load(f)
    good_features = []
    for pk, data in all_features.items():
        for feat in data.get("features", []):
            fn = feat.get("feature_name", feat.get("name", ""))
            ctx = feat.get("context", "")
            if fn and fn != "내용_구성" and len(ctx) > 10:
                good_features.append({"name": fn, "context": ctx[:80]})
    print(f"✅ 피처 로드: 의미 있는 피처 {len(good_features)}개")
else:
    good_features = []
    print("⚠️ 피처 파일 없음")

# ── 토픽 키워드 로드 ──────────────────────────────────────
PROMPT_FILES = {
    "익게2":     OUTPUT_DIR / "익게2_topics_for_prompt.json",
    "자유게시판": OUTPUT_DIR / "자유게시판_topics_for_prompt.json",
    "연애상담소": OUTPUT_DIR / "연애상담소_topics_for_prompt.json",
    "익게1":     OUTPUT_DIR / "익게1_topics_for_prompt.json",
}

trend_keywords = {}
for board, path in PROMPT_FILES.items():
    if path.exists():
        with open(path, "r", encoding="utf-8") as f:
            topics_data = json.load(f)
        top_topics = sorted(topics_data, key=lambda x: len(x.get("representatives", [])), reverse=True)[:5]
        keywords = []
        for topic in top_topics:
            kw = topic.get("keywords", [])
            if isinstance(kw, list): keywords.extend(kw[:8])
        trend_keywords[board] = list(dict.fromkeys(keywords))[:8]
        print(f"✅ {board}: {trend_keywords[board]}")

if not trend_keywords:
    trend_keywords = {"익게2": ["시험", "교수", "학점"], "자유게시판": ["취업", "대학원"]}

---
## 2. Gemini 헬퍼 + 생성 함수

In [ ]:
def call_gemini(prompt: str, temperature: float = 0.7) -> str:
    for attempt in range(3):
        try:
            response = gemini_model.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=temperature, max_output_tokens=2048,
                ),
            )
            return response.text.strip()
        except Exception as e:
            if "429" in str(e): time.sleep(15 * (attempt + 1))
            else: return ""
    return ""

def generate_post_gemini(board_name, keywords, factor_name, factor_desc=""):
    sampled_features = random.sample(good_features, min(5, len(good_features))) if good_features else []
    feature_sample = "\n".join([f"- {f['name']}: {f['context']}" for f in sampled_features])

    prompt = f"""당신은 서강대 커뮤니티 '서담'의 인기글 전문가입니다.
아래 조건으로 인기글을 작성하세요.

[게시판] {board_name}
[키워드] {', '.join(keywords)}
[팩터] {factor_name}: {factor_desc}
[피처 예시]\n{feature_sample}

형식:
<Thinking>전략 2~3문장</Thinking>
<Title>클릭유발 제목 (서담 말투)</Title>
<Content>실제 본문 (반말, 자연스럽게)</Content>"""

    result = call_gemini(prompt, 0.8)
    res_dict = {"board_name": board_name, "keywords": ", ".join(keywords), "factor": factor_name}
    for tag in ["Thinking", "Title", "Content"]:
        match = re.search(f'<{tag}>(.*?)</{tag}>', result, re.DOTALL)
        res_dict[f"generated_{tag.lower()}" if tag != "Thinking" else "thinking"] = match.group(1).strip() if match else ""
    res_dict["timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return res_dict

---
## 3. 신규 생성 실행 (중간 저장 포함)

In [ ]:
generated_posts = []
CACHE_FILE = OUTPUT_DIR / "generated_posts_gemini_temp.json"
SAVE_INTERVAL = 5

boards = list(trend_keywords.keys())
factors = factor_names[:3]

for b_idx, board in enumerate(boards):
    for f_idx, factor in enumerate(factors):
        print(f"🔧 [{board}] {factor} 생성 중...")
        res = generate_post_gemini(board, trend_keywords[board], factor, factor_descriptions.get(factor, ""))
        generated_posts.append(res)
        
        if len(generated_posts) % SAVE_INTERVAL == 0:
            with open(CACHE_FILE, "w", encoding="utf-8") as f:
                json.dump(generated_posts, f, ensure_ascii=False, indent=2)
            print(f"  💾 중간 저장됨 ({len(generated_posts)}건)")
        time.sleep(1)

final_csv = OUTPUT_DIR / "generated_posts_gemini.csv"
pd.DataFrame(generated_posts).to_csv(final_csv, index=False, encoding="utf-8-sig")
print(f"✅ 생성 완료! {final_csv}")

---
## 4. Phase 4 결과 재처리 (Refining)

Phase 4에서 Llama 모델이 생성한 초안을 로드하여 Gemini가 더 자연스럽게 교정합니다.

In [ ]:
PHASE4_CSV = OUTPUT_DIR / "generated_posts_v2.csv"
REFINED_CSV = OUTPUT_DIR / "generated_posts_refined.csv"

if PHASE4_CSV.exists():
    df_p4 = pd.read_csv(PHASE4_CSV)
    refined_data = []
    
    print(f"📝 Phase 4 데이터 {len(df_p4)}건 재처리 시작...")
    
    for idx, row in tqdm(df_p4.iterrows(), total=len(df_p4)):
        prompt = f"""아래는 인공지능이 작성한 커뮤니티 '서담'의 인기글 초안이야.
이 글을 실제 서강대 학생이 쓴 것처럼 훨씬 더 자연스럽고 생생하게 고쳐줘.

[초안 제목]: {row.get('generated_title', '')}
[초안 본문]:\n{row.get('generated_content', '')}

규칙:
- 말투는 친근한 반말로 수정 (서담 특유의 분위기 반영)
- 너무 정중하거나 설명조인 부분은 과감히 삭제
- <Title>, <Content> 태그로 감싸서 응답해줘."""
        
        res_text = call_gemini(prompt, 0.7)
        
        new_title = re.search(r'<Title>(.*?)</Title>', res_text, re.DOTALL)
        new_content = re.search(r'<Content>(.*?)</Content>', res_text, re.DOTALL)
        
        refined_data.append({
            "board_name": row.get('board_name'),
            "factor": row.get('factor'),
            "original_title": row.get('generated_title'),
            "refined_title": new_title.group(1).strip() if new_title else row.get('generated_title'),
            "refined_content": new_content.group(1).strip() if new_content else res_text,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })
        time.sleep(1)
    
    pd.DataFrame(refined_data).to_csv(REFINED_CSV, index=False, encoding="utf-8-sig")
    print(f"✅ 재처리 완료! {REFINED_CSV}")
    display(pd.DataFrame(refined_data).head())
else:
    print("⚠️ Phase 4 결과 파일이 없어 재처리를 건너뜁니다.")

---
## 5. 결과 상세 검토

In [ ]:
print("=== 신규 생성 샘플 ===")
for p in generated_posts[:2]:
    print(f"[제목] {p['generated_title']}\n[Thinking] {p['thinking']}\n{'-'*20}")

if 'refined_data' in locals() and refined_data:
    print("\n=== 재처리(Refined) 샘플 ===")
    for r in refined_data[:2]:
        print(f"[Original] {r['original_title']}\n[Refined] {r['refined_title']}\n{'-'*20}")